In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [8]:
# map strings to integers based on a provided vocabulary
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [11]:
# convert the created ids back to characters
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
# converts the characters into text
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
# each character will be represented as a vector of length 256
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # converts integer IDs into dense vectors of fixed size
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # recurrent layer that captures sequential patterns
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # outputs the logits for each word in the vocabulary
    self.dense = tf.keras.layers.Dense(vocab_size)

  # Defining the forward pass of the model
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# array of randomly sampled word indices based on the predicted logits
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"native town you enter'd like a post,\nAnd had no welcomes home: but he returns,\nSplitting the air wit"

Next Char Predictions:
 b"DAT eTFtCByFZe kqTDmCZ!mHQQJHLCcnshsnpf'izlC&nEPQ3V\njfQ.gzX,Me.YqEwsUvXHw3iv,wZoQmBYsi-q3R\nWXQaLCKZw"


In [31]:
# calculates the cross-entropy loss between the predicted probabilities and the true labels.
# from logits is true as we are directly working with raw output logits which have not been put through softmax layer
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189846, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

66.01263

In [34]:
model.compile(optimizer='adam', loss=loss)

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [49]:
EPOCHS = 30

In [50]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 11s 54ms/step - loss: 1.1310
Epoch 2/40
172/172 [==============================] - 11s 55ms/step - loss: 1.0899
Epoch 3/40
172/172 [==============================] - 11s 55ms/step - loss: 1.0444
Epoch 4/40
172/172 [==============================] - 11s 56ms/step - loss: 0.9973
Epoch 5/40
172/172 [==============================] - 12s 57ms/step - loss: 0.9478
Epoch 6/40
172/172 [==============================] - 12s 58ms/step - loss: 0.8955
Epoch 7/40
172/172 [==============================] - 12s 59ms/step - loss: 0.8440
Epoch 8/40
172/172 [==============================] - 12s 59ms/step - loss: 0.7918
Epoch 9/40
172/172 [==============================] - 12s 57ms/step - loss: 0.7427
Epoch 10/40
172/172 [==============================] - 12s 57ms/step - loss: 0.6966
Epoch 11/40
172/172 [==============================] - 12s 58ms/step - loss: 0.6554
Epoch 12/40
172/172 [==============================] - 12s 59ms/step - loss: 0.6184
E

In [51]:
# represents a single step of text generation using a given model
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    #  A parameter that controls the randomness of the generated text.
    # Higher values (e.g., 1.0) result in more random outputs,
    # while lower values (e.g., 0.5) make the outputs more deterministic.
    self.temperature = temperature
    # model we created earlier
    self.model = model
    # both functions defined earlier
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  # generates the next character given an input sequence.
  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [52]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [53]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The prettiest love I can be honour in our hearts.
What, unburith age? what statut is most of all.

PETRUCHIO:
Signior Hortensio,
O Ratcliff, I fear, when weep your power.

Post:
An id the heads of the cause. I'll to die a smoot
That hunger for his death-bed lived,
Forsworn to fall their heirs, the brank'd souls
And all sister's pursuins to be married; and
By calming title grung encounters at me.
More than terror; and the wold with us,
Or see have you are all undone: upon this good disdail
For, thou bold love, and may look on the poor sould,
And darts his mighty seventions and grievous
plainly so likely unmon'd, a very view treach:
Shall then the broker inferial shadows;
Which sorrow will shed from Claudio :
desperated the fairest than an ague-face to a
hog sweet son.

LUCIO:
This may part with the king.

CLIFFORD:
What, not a word? yes, you have unnatural,
Than to demand disdraintually the stones;
And low with lest should close it to your bosom's tribune.

KING EDWARD IV:
Make w

In [54]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThere is my banish'd, hence, awake, mat\nDrath in the holy trads, fair quarrels; and their beauteous lies\nThe might child that have beseemed my feed,\nMay to dinne hence perfumed for the moon.\n\nPETRUCHIO:\nSo I be gone?\nWhat is the book on his king.\n\nWARWICK:\nI jush'd his law; as they will look upon,\nBy help to our mistress for their dusty there.\nGo, some sand a bark, our case, and they smart\nTo hear them toll. This man, Aufidius,\nWell-minusters; though it do some prive\nThan this unfold some of it.\n\nFirst Of the devil hath more grief to pieces.\n\nLUCENTIO:\nAnd yet I would he had no tongue,\nwhich in the case of the remedy; and all o'er down\nEncounalement, who's near and him;\nAnd fearless father to do it overal periul.\n\nhigh, for 'tis a sore man I speak of nature,\nSo long than very weak cried, as Camillo.\n\nCAMILLO:\nBy miserainess! farewell.\nBut of this noble uncles, God pave boldly coll\nMakes me with the easy grove to my wife.\n\nLUCENTIO:

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [43]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Petruchio go wind thy wife.

ESCALUS:
What is she nam?

BENVOLIO:
Am sturb'd, I would say as he str


In [44]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [45]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [48]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1556
Epoch 1 Batch 50 Loss 2.0626
Epoch 1 Batch 100 Loss 1.9674
Epoch 1 Batch 150 Loss 1.8770

Epoch 1 Loss: 1.9862
Time taken for 1 epoch 12.97 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7924
Epoch 2 Batch 50 Loss 1.7125
Epoch 2 Batch 100 Loss 1.7081
Epoch 2 Batch 150 Loss 1.6625

Epoch 2 Loss: 1.7115
Time taken for 1 epoch 11.89 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5895
Epoch 3 Batch 50 Loss 1.5913
Epoch 3 Batch 100 Loss 1.5480
Epoch 3 Batch 150 Loss 1.5403

Epoch 3 Loss: 1.5504
Time taken for 1 epoch 20.47 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4889
Epoch 4 Batch 50 Loss 1.4299
Epoch 4 Batch 100 Loss 1.4482
Epoch 4 Batch 150 Loss 1.4233

Epoch 4 Loss: 1.4532
Time taken for 1 epoch 11.33 sec
_____________________________________________________________________